# Roadmap

$$ \text{Raster}\xleftarrow{\text{1ms bin}} \text{Relative Response: } T \times (N * B)$$
<br>
$$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\downarrow{\frac{\Sigma{\text{ (columns: T)}}} {\text{T in E}}} $$
<br>
$$ PSTH $$
T = trials&emsp;&emsp;N = neurons&emsp;&emsp;B = bins&emsp;&emsp;E = events

# JSON Files
## Matlab
```
% get file name and path
% open file with fopen and load in data
% close file and use jsondecode on the data read in from file
example_data = jsondecode(str_data);
```

## Python

```
import json
# open file
# Use load or other json functions to read in data
example_data = json.load(file)
```

# JSON Example
```json
{
    "events": {
        "event_3": [
            138.33675
        ],
         "event_6": [
            42.74935
        ]
    }
}
```
<br>
```py
dict['events'] = ['event_1', 'event_4']
```

In [2]:
import json

with open('hw1_example.json') as input:
    example_data = json.load(input)

In [3]:
    print(example_data.keys())
    print(example_data['events'].keys())
    print(example_data['neurons'].keys())

dict_keys(['events', 'neurons'])
dict_keys(['event_1', 'event_4'])
dict_keys(['sig001a', 'sig007c'])


# Psuedocode
```py
# Go through collection of bin sizes to create PSTH
for bin_size in bin_collection
    window_range = pre_time:bin_size:post_time
```

```py
# Go through event keys and grab event timestamps
    for curr_event in events:
        event_ts = example_data['events'][curr_event]
```

```py
# Go through neurons and grab neuron spike times
        for curr_neuron in neurons:
            neuron_ts = example_data['neurons'][curr_neuron]
```

```py
# Go through trials and grab trial timestamp to create histogram
            for curr_trial in curr_event
                offset_ts = neuron_ts - curr_trial
                binned_response = histohtam(offset_ts, window_range)
                response[trial_i, unit_start:unit_end] = binned_response
```

# Python Raster Sidenote
In order to plot the raster graphs in matplotlib, map 1s to the relative spike time and 0s to NaN.  
## ex  
Parameters: 1ms bin size from 0ms to 5ms
$$\text{0 0 1 0 1} \rightarrow \text{NaN NaN .003 NaN .005}$$

![Sig 007c Event 4 Raster](images/sig007c_event_4_0.001ms_raster.png)